In [1]:
import numpy as np
import pandas as pd
import tqdm

import peyes
import analysis.utils as u

## Load Data

In [2]:
dataset = peyes.datasets.lund2013(directory=u.DATASETS_DIR, save=False, verbose=True)
dataset.head()

,trial_id,subject_id,stimulus_type,stimulus_name,t,x,y,pupil,pixel_size,viewer_distance,MN,RA
0,1,TH20,moving_dot,1,0.0,123.2532,22.6264,NaN,0.037824,67.0,1.0,1.0
1,1,TH20,moving_dot,1,2.0,123.5395,22.9064,NaN,0.037824,67.0,1.0,1.0
2,1,TH20,moving_dot,1,4.0,123.2230,21.9909,NaN,0.037824,67.0,1.0,1.0
3,1,TH20,moving_dot,1,6.0,123.1883,21.7740,NaN,0.037824,67.0,1.0,1.0
4,1,TH20,moving_dot,1,8.0,125.0540,21.1805,NaN,0.037824,67.0,1.0,1.0


In [3]:
VIEWER_DISTANCE_CM = dataset["viewer_distance"].unique()[0]
PIXEL_SIZE_CM = dataset["pixel_size"].unique()[0]

PIXEL_SIZE_DEG = peyes._utils.pixel_utils.pixels_to_visual_angle(1, VIEWER_DISTANCE_CM, PIXEL_SIZE_CM)
DEG_IN_PIXEL = peyes._utils.pixel_utils.visual_angle_to_pixels(1, VIEWER_DISTANCE_CM, PIXEL_SIZE_CM)

print(f"Viewer distance:\t\t{10 * VIEWER_DISTANCE_CM}mm")
print(f"Pixel size:\t\t\t\t{10 * PIXEL_SIZE_CM:.3f}mm")
print(f"Pixel size:\t\t\t\t{PIXEL_SIZE_DEG:.3f}° (DVA)")
print(f"1° (DVA) in pixels:\t\t{DEG_IN_PIXEL:.2f}px")

Viewer distance:		670.0mm
Pixel size:				0.378mm
Pixel size:				0.032° (DVA)
1° (DVA) in pixels:		30.92px


## Event-Features Statistics
### (Image Stimuli Only)

In [4]:
QUANTILES = [0.001, 0.01, 0.025, 0.05, 0.5, 0.95, 0.975, 0.99, 0.999]
EVENT_PROPORTION = 95   # 95% of events
FEATURE_CI = 95         # 95% confidence interval for feature statistic

def labels_to_events(dataframe: pd.DataFrame):
    trial_ids = dataframe[peyes.constants.TRIAL_ID_STR].unique()
    annotators = set(dataframe.columns).intersection(set([ann for annotators in u.DATASET_ANNOTATORS.values() for ann in annotators]))
    event_dict = {}
    for i, trial_id in tqdm.tqdm(enumerate(trial_ids), total=len(trial_ids)):
        trial_data = dataframe[dataframe["trial_id"] == trial_id]
        stim_type, stim_name = trial_data[[peyes.constants.STIMULUS_TYPE_STR, peyes.constants.STIMULUS_NAME_STR]].values[0]
        t = trial_data[peyes.constants.T].values
        x = trial_data[peyes.constants.X].values
        y = trial_data[peyes.constants.Y].values
        pupil = trial_data[peyes.constants.PUPIL].values
        ps = trial_data[peyes.constants.PIXEL_SIZE_STR].values[0]
        vd = trial_data[peyes.constants.VIEWER_DISTANCE_STR].values[0]
        for annotator in annotators:
            evnts = peyes.create_events(
                labels=trial_data[annotator].values,
                t=t, x=x, y=y, pupil=pupil, pixel_size=ps, viewer_distance=vd,
            )
            evnts = pd.Series(evnts, name=(trial_id, annotator))
            event_dict[(trial_id, stim_type, stim_name, annotator)] = evnts
    event_df = pd.DataFrame(event_dict).T.dropna(axis=0, how='all')
    event_df.index.names = [
        peyes.constants.TRIAL_ID_STR, peyes.constants.STIMULUS_TYPE_STR, peyes.constants.STIMULUS_NAME_STR, "annotator"
    ]
    return event_df


def events_df_to_series(events_df: pd.DataFrame, min_num_samples: int = 2) -> pd.Series:
    events_as_series = events_df.groupby(
        level=np.arange(events_df.index.nlevels).tolist()
    ).apply(
        lambda sub: pd.Series(sub.values.flatten()).dropna()
    )
    events_as_series = events_as_series[events_as_series.map(lambda x: x.num_samples >= min_num_samples)]
    return events_as_series

In [5]:
events = labels_to_events(dataset)
events_series = events_df_to_series(events, min_num_samples=2)

image_events = events.xs(peyes.constants.IMAGE_STR, level=peyes.constants.STIMULUS_TYPE_STR)
image_events_series = events_series.xs(peyes.constants.IMAGE_STR, level=peyes.constants.STIMULUS_TYPE_STR)

100%|██████████| 63/63 [00:00<00:00, 102.76it/s]


#### Feature Quantiles
For each (label, rater) pair, we calculate the total number of events and their distribution of duration, dispersion, max velocity and min velocity.  
We present this data as a table specifying the quantiles for 0.1%, 1%, 2.5%, 5%, 50%, 95%, 97.5%, 99%, 99.9%

In [6]:
event_quantiles = {}

for label in peyes._DataModels.EventLabelEnum.EventLabelEnum:
    if label.value == 0:
        continue
    series = image_events_series[image_events_series.apply(lambda x: x.label == label.value)]
    
    # COUNTS
    counts = series.groupby(level="annotator").size().rename("")
    counts.loc["both"] = counts.sum()
    
    # DURATIONS
    dur_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.duration).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    dur_percentiles.loc["both"] = [series.apply(lambda event: event.duration).quantile(q) for q in QUANTILES]
    
    if label == peyes._DataModels.EventLabelEnum.EventLabelEnum.BLINK:
        stats = pd.concat([counts, dur_percentiles], keys=["counts", "durations"], axis=1)
        event_quantiles[label.name] = stats
        continue
    
    # DISPERSIONS
    disp_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.dispersion).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    disp_percentiles.loc["both"] = [series.apply(lambda event: event.dispersion).quantile(q) for q in QUANTILES]
    
    # PEAK VELOCITIES
    peak_vel_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.peak_velocity).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    peak_vel_percentiles.loc["both"] = [series.apply(lambda event: event.peak_velocity).quantile(q) for q in QUANTILES]
    
    # MIN VELOCITIES
    min_vel_percentiles = pd.concat([
        series.groupby(level="annotator").apply(
            lambda sub_series: sub_series.apply(lambda event: event.min_velocity).quantile(q)
        ).rename(f"{100 * q}%") for q in QUANTILES
    ], axis=1)
    min_vel_percentiles.loc["both"] = [series.apply(lambda event: event.min_velocity).quantile(q) for q in QUANTILES]
    
    stats = pd.concat(
        [counts, dur_percentiles, disp_percentiles, peak_vel_percentiles, min_vel_percentiles],
        keys=["counts", "durations", "dispersions", "peak_velocities", "min_velocities"],
        axis=1
    )
    event_quantiles[label.name] = stats
    
event_quantiles = pd.concat(event_quantiles, axis=0, names=[peyes.constants.EVENT_STR, "annotator"])

print("##  IMPORTANT NOTE  ##")
print("Not all features are applicable to all events (e.g., blinks have no velocity), so some statistics may be missing.")
event_quantiles

##  IMPORTANT NOTE  ##
Not all features are applicable to all events (e.g., blinks have no velocity), so some statistics may be missing.


counts   durations                                    \
                                       0.1%       1.0%        2.5%       5.0%   
event          annotator                                                        
FIXATION       MN           404    6.809030   32.43285   78.308300   96.01135   
               RA           563   13.867562   50.48652   80.218900  100.02150   
               both         967    5.999000   46.00866   80.011350   96.02080   
SACCADE        MN           377    6.003632   10.00028   10.010000   12.00000   
               RA           552    8.000551    9.99608   11.998100   12.01540   
               both         929    6.007496    9.99856   10.011000   12.00240   
PSO            MN           312    3.995244    5.99811    6.000000    7.99620   
               RA           418    4.001000    4.99817    6.004700    7.99900   
               both         730    3.996916    4.99829    6.001000    7.99900   
SMOOTH_PURSUIT MN             3  152.582114  157.40714  165.448850  178.85170   
               RA            30   54.529464   59.23164   67.068600  100.83020   
               both          33   54.583512   59.77212   68.419800  110.43260   
BLINK          MN            22   90.129727   91.07227   92.643175   97.86260   
               RA            23   94.046714   94.24214   94.567850   99.51400   
               both          45   90.201000   91.78500   94.123700   95.01200   

                                                                       \
                             50.0%      95.0%       97.5%       99.0%   
event          annotator                                                
FIXATION       MN         220.0500  521.03995  681.388700  1173.03150   
               RA         226.0500  490.70040  603.061150  1075.83428   
               both       224.9990  508.56940  626.323700  1160.43630   
SACCADE        MN          28.0010   52.00820   57.213200    66.01340   
               RA          30.0175   55.45190   60.012825    72.44506   
               both        30.0020   54.00960   60.008800    68.88444   
PSO            MN          21.9990   50.00945   56.916825    64.45832   
               RA          19.9985   40.01015   50.004025    55.00066   
               both        20.0050   46.00855   54.020625    60.00000   
SMOOTH_PURSUIT MN         420.1030  502.90570  507.505850   510.26594   
               RA         268.0605  658.93170  691.294600   702.61504   
               both       270.0600  650.52960  689.342800   701.83432   
BLINK          MN         319.0690  520.01890  574.276600   610.19164   
               RA         329.9960  623.73790  839.885800   993.70012   
               both       322.0710  617.33680  633.735400   892.91548   

                                       ... peak_velocities min_velocities  \
                                99.9%  ...           99.9%           0.1%   
event          annotator               ...                                  
FIXATION       MN         3530.437680  ...      170.529352       0.035566   
               RA         3175.250348  ...      164.736555       0.036727   
               both       4421.096796  ...      170.223810       0.032622   
SACCADE        MN          110.507248  ...      171.630742       1.401812   
               RA          107.196919  ...      171.702116       1.740005   
               both        116.173856  ...      172.872905       1.658814   
PSO            MN           69.998689  ...      151.529737       1.357170   
               RA           65.830583  ...      145.134590       0.679208   
               both         69.999542  ...      148.663984       0.773734   
SMOOTH_PURSUIT MN          511.921994  ...      161.692930       1.456277   
               RA          709.407304  ...      169.462619       0.371445   
               both        709.329232  ...      169.438952       0.371848   
BLINK          MN          631.740664  ...             NaN            NaN   
               RA  

In [55]:
event_quantiles.xs('min_velocities', axis=1)

0.1%      1.0%      2.5%      5.0%      50.0%  \
event          annotator                                                      
FIXATION       MN         0.035566  0.071895  0.180977  0.253975   1.027135   
               RA         0.036727  0.073450  0.151494  0.253691   0.972174   
               both       0.032622  0.071816  0.154578  0.253731   0.991963   
SACCADE        MN         1.401812  2.436304  5.208581  7.240570  29.913226   
               RA         1.740005  3.078073  4.455919  6.470810  27.161543   
               both       1.658814  2.640067  4.502300  6.718161  28.440431   
PSO            MN         1.357170  1.811174  2.660823  3.481902  14.781779   
               RA         0.679208  1.611016  1.994167  2.682294  12.012524   
               both       0.773734  1.683227  2.176975  3.060359  13.242168   
SMOOTH_PURSUIT MN         1.456277  1.467460  1.486098  1.517162   2.076302   
               RA         0.371445  0.406507  0.464942  0.529556   1.172636   
               both       0.371848  0.410537  0.475017  0.538780   1.220562   
BLINK          MN              NaN       NaN       NaN       NaN        NaN   
               RA              NaN       NaN       NaN       NaN        NaN   
               both            NaN       NaN       NaN       NaN        NaN   

                              95.0%      97.5%       99.0%       99.9%  
event          annotator                                                
FIXATION       MN          2.989608   4.132497    8.236544   23.165551  
               RA          2.898477   3.503758    4.270721   19.199061  
               both        2.958909   3.633974    5.312801   22.665301  
SACCADE        MN         84.466290  96.432994  106.657180  132.842196  
               RA         76.615685  89.692423  104.063074  131.803931  
               both       80.263971  93.290203  105.309931  138.093096  
PSO            MN         43.233754  52.470840   63.514153   71.494832  
               RA         34.239748  40.672586   46.224312   79.180612  
               both       38.324552  45.069794   57.817433   76.091224  
SMOOTH_PURSUIT MN          2.488836   2.511755    2.525506    2.533757  
               RA          7.388151   8.877995   10.575343   11.593751  
               both        7.249214   8.585349   10.458284   11.582045  
BLINK          MN               NaN        NaN         NaN         NaN  
               RA               NaN        NaN         NaN         NaN  
               both             NaN        NaN         NaN         NaN

#### Threshold Cutoffs
We specify the % of events that pass the selected thresholds (see table of thresholds)

In [72]:
thresholds = pd.DataFrame.from_dict({
    1: {
        ('duration', 'min'): 50,        # Andersson et. al (2016) used 55ms
        ('duration', 'max'): 5000,
        ('dispersion', 'max'): 2.7      # used by Andersson et. al (2016) - I-DT
    },
    2: {
        ('duration', 'min'): 6,
        ('duration', 'max'): 150,
        ('dispersion', 'min'): 2.7,     # used by Andersson et. al (2016) - I-DT
        ('min_velocity', 'min'): 45     # used by Andersson et. al (2016) - I-VT
    },
    3: {
        ('duration', 'min'): 2,
        ('duration', 'max'): 60,
    },
    4: {
        ('duration', 'min'): 50,
        ('duration', 'max'): 5000,
        ('dispersion', 'min'): 2.7,     # used by Andersson et. al (2016) - I-DT
        ('min_velocity', 'min'): 26     # used by Komogortsev & Karpov (2013) - I-VVT
    },
    5: {
        ('duration', 'min'): 50,
        ('duration', 'max'): 2000,
    },
}).sort_index().T
thresholds

dispersion      duration       min_velocity
         max  min      max   min          min
1        2.7  NaN   5000.0  50.0          NaN
2        NaN  2.7    150.0   6.0         45.0
3        NaN  NaN     60.0   2.0          NaN
4        NaN  2.7   5000.0  50.0         26.0
5        NaN  NaN   2000.0  50.0          NaN

In [71]:
fixs_dur_between_50_55 = image_events_series[image_events_series.apply(lambda x: x.label == 1)].map(lambda x: 50 <= x.duration <= 55).groupby(level=[0, 1, 2]).sum()

print(f"Andersson et al. (2016) used a minimum duration threshold of 55ms, but there are {fixs_dur_between_50_55.sum()} additional fixations that fall between 50-55ms, and 50ms is a physiologically plausible value.")

Andersson et al. (2016) used a minimum duration threshold of 55ms, but there are 2 additional fixations that fall between 50-55ms, and 50ms is a physiologically plausible value.


In [73]:
success_rates = {}

for idx in sorted(thresholds.index.unique()):
    label = peyes.parse_label(idx)
    series = image_events_series[image_events_series.apply(lambda x: x.label == idx)]
    counts = series.groupby(level="annotator").size().rename("total")
    counts.loc["both"] = counts.sum()
    success_rates[(label.name, "counts", '')] = counts
    
    for feat, op in thresholds.columns:
        if np.isnan(thresholds.loc[idx, (feat, op)]):
            continue
        if op == "min":
            success = series.apply(lambda event: getattr(event, feat) >= thresholds.loc[idx, (feat, op)])
        elif op == "max":
            success = series.apply(lambda event: getattr(event, feat) <= thresholds.loc[idx, (feat, op)])
        else:
            raise ValueError(f"Invalid operation: {op}")
        success = success.groupby(level="annotator").sum()
        success.loc["both"] = success.sum()
        success_rates[(label.name, feat, op)] = (100 * success / counts).rename((feat, op))

success_rates = pd.concat(success_rates, axis=1).stack(0, future_stack=True).reorder_levels([1, 0])
success_rates = success_rates.reindex(event_quantiles.index)
success_rates = success_rates[sorted(success_rates.columns)]

print("The following values show what % of events meet the thresholds specified in the above code-box.")
success_rates

The following values show what % of events meet the thresholds specified in the above code-box.


counts dispersion               duration              \
                                       max        min         max         min   
event          annotator                                                        
FIXATION       MN           404  98.019802        NaN  100.000000   98.267327   
               RA           563  99.111901        NaN  100.000000   98.934281   
               both         967  98.655636        NaN  100.000000   98.655636   
SACCADE        MN           377        NaN  79.575597  100.000000  100.000000   
               RA           552        NaN  79.166667  100.000000  100.000000   
               both         929        NaN  79.332616  100.000000  100.000000   
PSO            MN           312        NaN        NaN   98.076923  100.000000   
               RA           418        NaN        NaN   99.760766  100.000000   
               both         730        NaN        NaN   99.041096  100.000000   
SMOOTH_PURSUIT MN             3        NaN  66.666667  100.000000  100.000000   
               RA            30        NaN  20.000000  100.000000  100.000000   
               both          33        NaN  24.242424  100.000000  100.000000   
BLINK          MN            22        NaN        NaN  100.000000  100.000000   
               RA            23        NaN        NaN  100.000000  100.000000   
               both          45        NaN        NaN  100.000000  100.000000   

                         min_velocity  
                                  min  
event          annotator               
FIXATION       MN                 NaN  
               RA                 NaN  
               both               NaN  
SACCADE        MN           27.055703  
               RA           20.833333  
               both         23.358450  
PSO            MN                 NaN  
               RA                 NaN  
               both               NaN  
SMOOTH_PURSUIT MN            0.000000  
               RA            0.000000  
               both          0.000000  
BLINK          MN                 NaN  
               RA                 NaN  
               both               NaN

========================================================================================================================

### Fixation Features

**DURATIONS**

In [ ]:
fix_dur_percentiles = [
    fix_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda fix: fix.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
fix_dur_percentiles = pd.concat([fix_counts, *fix_dur_percentiles], axis=1)
fix_dur_percentiles.loc["both"] = [
    fix_counts.sum(),
    *[fix_series.apply(lambda fix: fix.duration).quantile(q) for q in QUANTILES]
]

print("FIXATION DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {fix_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {fix_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{fix_dur_percentiles.loc['both', '2.5%']:.2f}, {fix_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
fix_dur_percentiles

**DISPERSIONS**

In [ ]:
fix_disp_percentiles = [
    fix_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda fix: fix.dispersion).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
fix_disp_percentiles = pd.concat([fix_counts, *fix_disp_percentiles], axis=1)
fix_disp_percentiles.loc["both"] = [
    fix_counts.sum(),
    *[fix_series.apply(lambda fix: fix.dispersion).quantile(q) for q in QUANTILES]
]

print("FIXATION DISPERSIONS (DVA)")
print(f"{EVENT_PROPORTION}% have dispersion <= {fix_disp_percentiles.loc['both', '95.0%']:.2f}°")
print(f"{FEATURE_CI}% CI for dispersion is [{fix_disp_percentiles.loc['both', '2.5%']:.2f}, {fix_disp_percentiles.loc['both', '97.5%']:.2f}]°\n")
fix_disp_percentiles

**PEAK VELOCITIES**

In [ ]:
fix_peak_vel_percentiles = [
    fix_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda fix: fix.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
fix_peak_vel_percentiles = pd.concat([fix_counts, *fix_peak_vel_percentiles], axis=1)
fix_peak_vel_percentiles.loc["both"] = [
    fix_counts.sum(),
    *[fix_series.apply(lambda fix: fix.peak_velocity).quantile(q) for q in QUANTILES]
]

print("FIXATION PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity <= {fix_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{fix_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {fix_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
fix_peak_vel_percentiles

### Saccade Features

In [ ]:
sac_series = image_events_series[image_events_series.apply(lambda x: x.label == 2)]
sac_counts = sac_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [ ]:
sac_dur_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_dur_percentiles = pd.concat([sac_counts, *sac_dur_percentiles], axis=1)
sac_dur_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.duration).quantile(q) for q in QUANTILES]
]

print("SACCADE DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {sac_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {sac_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{sac_dur_percentiles.loc['both', '2.5%']:.2f}, {sac_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
sac_dur_percentiles

**DISPERSIONS**

In [ ]:
sac_disp_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.dispersion).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_disp_percentiles = pd.concat([sac_counts, *sac_disp_percentiles], axis=1)
sac_disp_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.dispersion).quantile(q) for q in QUANTILES]
]

print("SACCADE DISPERSIONS (DVA)")
print(f"{EVENT_PROPORTION}% have dispersion >= {sac_disp_percentiles.loc['both', '5.0%']:.2f}°")
print(f"{FEATURE_CI}% CI for dispersion is [{sac_disp_percentiles.loc['both', '2.5%']:.2f}, {sac_disp_percentiles.loc['both', '97.5%']:.2f}]°\n")
sac_disp_percentiles

**MIN VELOCITIES**

In [ ]:
sac_min_vel_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.min_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_min_vel_percentiles = pd.concat([sac_counts, *sac_min_vel_percentiles], axis=1)
sac_min_vel_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.min_velocity).quantile(q) for q in QUANTILES]
]

print("SACCADE MIN VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have min velocity >= {sac_min_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for min velocity is [{sac_min_vel_percentiles.loc['both', '2.5%']:.2f}, {sac_min_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sac_min_vel_percentiles

**PEAK VELOCITIES**

In [ ]:
sac_peak_vel_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_peak_vel_percentiles = pd.concat([sac_counts, *sac_peak_vel_percentiles], axis=1)
sac_peak_vel_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.peak_velocity).quantile(q) for q in QUANTILES]
]

print("SACCADE PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity >= {sac_peak_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have peak velocity <= {sac_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{sac_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {sac_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sac_peak_vel_percentiles

**AMPLITUDES**

In [ ]:
sac_amp_percentiles = [
    sac_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sac: sac.amplitude).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sac_amp_percentiles = pd.concat([sac_counts, *sac_amp_percentiles], axis=1)
sac_amp_percentiles.loc["both"] = [
    sac_counts.sum(),
    *[sac_series.apply(lambda sac: sac.amplitude).quantile(q) for q in QUANTILES]
]

print("SACCADE AMPLITUDES (DVA)")
print(f"{EVENT_PROPORTION}% have amplitude >= {sac_amp_percentiles.loc['both', '5.0%']:.2f}°")
print(f"{EVENT_PROPORTION}% have amplitude <= {sac_amp_percentiles.loc['both', '95.0%']:.2f}°")
print(f"{FEATURE_CI}% CI for amplitude is [{sac_amp_percentiles.loc['both', '2.5%']:.2f}, {sac_amp_percentiles.loc['both', '97.5%']:.2f}]°\n")
sac_amp_percentiles

### Smooth Pursuit Features

In [ ]:
sp_series = image_events_series[image_events_series.apply(lambda x: x.label == 4)]
sp_counts = sp_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [ ]:
sp_dur_percentiles = [
    sp_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sp: sp.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sp_dur_percentiles = pd.concat([sp_counts, *sp_dur_percentiles], axis=1)
sp_dur_percentiles.loc["both"] = [
    sp_counts.sum(),
    *[sp_series.apply(lambda sp: sp.duration).quantile(q) for q in QUANTILES]
]

print("SMOOTH PURSUIT DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {sp_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {sp_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{sp_dur_percentiles.loc['both', '2.5%']:.2f}, {sp_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
sp_dur_percentiles

**MIN VELOCITIES**

In [ ]:
sp_min_vel_percentiles = [
    sp_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sp: sp.min_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sp_min_vel_percentiles = pd.concat([sp_counts, *sp_min_vel_percentiles], axis=1)
sp_min_vel_percentiles.loc["both"] = [
    sp_counts.sum(),
    *[sp_series.apply(lambda sp: sp.min_velocity).quantile(q) for q in QUANTILES]
]

print("SMOOTH PURSUIT MIN VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have min velocity >= {sp_min_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have min velocity <= {sp_min_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for min velocity is [{sp_min_vel_percentiles.loc['both', '2.5%']:.2f}, {sp_min_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sp_min_vel_percentiles

**PEAK VELOCITIES**

In [ ]:
sp_peak_vel_percentiles = [
    sp_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda sp: sp.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
sp_peak_vel_percentiles = pd.concat([sp_counts, *sp_peak_vel_percentiles], axis=1)
sp_peak_vel_percentiles.loc["both"] = [
    sp_counts.sum(),
    *[sp_series.apply(lambda sp: sp.peak_velocity).quantile(q) for q in QUANTILES]
]

print("SMOOTH PURSUIT PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity >= {sp_peak_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have peak velocity <= {sp_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{sp_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {sp_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
sp_peak_vel_percentiles

### PSO Features

In [ ]:
pso_series = image_events_series[image_events_series.apply(lambda x: x.label == 3)]
ps_counts = pso_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [ ]:
pso_dur_percentiles = [
    pso_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda pso: pso.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
pso_dur_percentiles = pd.concat([ps_counts, *pso_dur_percentiles], axis=1)
pso_dur_percentiles.loc["both"] = [
    ps_counts.sum(),
    *[pso_series.apply(lambda pso: pso.duration).quantile(q) for q in QUANTILES]
]

print("PURSUIT SMOOTHING DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {pso_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {pso_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{pso_dur_percentiles.loc['both', '2.5%']:.2f}, {pso_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
pso_dur_percentiles

**MIN VELOCITIES**

In [ ]:
pso_min_vel_percentiles = [
    pso_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda pso: pso.min_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
pso_min_vel_percentiles = pd.concat([ps_counts, *pso_min_vel_percentiles], axis=1)
pso_min_vel_percentiles.loc["both"] = [
    ps_counts.sum(),
    *[pso_series.apply(lambda pso: pso.min_velocity).quantile(q) for q in QUANTILES]
]

print("PSO MIN VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have min velocity >= {pso_min_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have min velocity <= {pso_min_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for min velocity is [{pso_min_vel_percentiles.loc['both', '2.5%']:.2f}, {pso_min_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
pso_min_vel_percentiles

**PEAK VELOCITIES**

In [ ]:
pso_peak_vel_percentiles = [
    pso_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda pso: pso.peak_velocity).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
pso_peak_vel_percentiles = pd.concat([ps_counts, *pso_peak_vel_percentiles], axis=1)
pso_peak_vel_percentiles.loc["both"] = [
    ps_counts.sum(),
    *[pso_series.apply(lambda pso: pso.peak_velocity).quantile(q) for q in QUANTILES]
]

print("PSO PEAK VELOCITIES (DVA/s)")
print(f"{EVENT_PROPORTION}% have peak velocity >= {pso_peak_vel_percentiles.loc['both', '5.0%']:.2f}°/s")
print(f"{EVENT_PROPORTION}% have peak velocity <= {pso_peak_vel_percentiles.loc['both', '95.0%']:.2f}°/s")
print(f"{FEATURE_CI}% CI for peak velocity is [{pso_peak_vel_percentiles.loc['both', '2.5%']:.2f}, {pso_peak_vel_percentiles.loc['both', '97.5%']:.2f}]°/s\n")
pso_peak_vel_percentiles

### Blink Features

In [ ]:
bl_series = image_events_series[image_events_series.apply(lambda x: x.label == 5)]
bl_counts = bl_series.groupby(level="annotator").size().rename("total")

**DURATIONS**

In [ ]:
bl_dur_percentiles = [
    bl_series.groupby(level="annotator").apply(
        lambda sub_series: sub_series.apply(lambda bl: bl.duration).quantile(q)
    ).rename(f"{100*q}%") for q in QUANTILES
]
bl_dur_percentiles = pd.concat([bl_counts, *bl_dur_percentiles], axis=1)
bl_dur_percentiles.loc["both"] = [
    bl_counts.sum(),
    *[bl_series.apply(lambda bl: bl.duration).quantile(q) for q in QUANTILES]
]

print("BLINK DURATIONS (ms)")
print(f"{EVENT_PROPORTION}% have duration >= {bl_dur_percentiles.loc['both', '5.0%']:.2f}ms")
print(f"{EVENT_PROPORTION}% have duration <= {bl_dur_percentiles.loc['both', '95.0%']:.2f}ms")
print(f"{FEATURE_CI}% CI for duration is [{bl_dur_percentiles.loc['both', '2.5%']:.2f}, {bl_dur_percentiles.loc['both', '97.5%']:.2f}]ms\n")
bl_dur_percentiles